In [ ]:
#
# hw7pr3:  titanic-passenger clasification via nearest neighbors
#

In [1]:
# libraries!
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

In [2]:
# let's read in our digits data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
filename = 'titanic4.csv'
df = pd.read_csv(filename, header=0)   # read the file w/header row #0
print(f"{filename} : file read!")

titanic4.csv : file read!


In [3]:
#
# before converting to a numpy array, let's use some pandas data-viewing and -cleaning...
#

df.head()   # this shows the first few rows, to get a sense of the data

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3,-1,"Ali, Mr. Ahmed",male,24.0,0,0,SOTON/O.Q. 3101311,7.0500,NaN,S,NaN,NaN,NaN
1,3,-1,"Ali, Mr. William",male,25.0,0,0,SOTON/O.Q. 3101312,7.0500,NaN,S,NaN,79.0,Argentina
2,3,-1,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN,NaN,"Lower Clapton, Middlesex or Erdington, Birmingham"
3,3,-1,"Allum, Mr. Owen George",male,18.0,0,0,2223,8.3000,NaN,S,NaN,259.0,"Windsor, England New York, NY"
4,3,-1,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,19.0,1,0,350046,7.8542,NaN,S,16,NaN,NaN


In [4]:
#
# this df.info() call will show the different features (columns), their types, and how many non-null there are ...
#

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [5]:
# ok!  let's drop the columns with too few items: cabin, boat, body, and home.dest
df = df.drop('cabin', axis=1)  # axis = 1 indicates we want to drop a column, not a row
df = df.drop('boat', axis=1)  
df = df.drop('body', axis=1)  
df = df.drop('home.dest', axis=1)  
df.info()  # re-look at the data ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   name      1309 non-null   object 
 3   sex       1309 non-null   object 
 4   age       1046 non-null   float64
 5   sibsp     1309 non-null   int64  
 6   parch     1309 non-null   int64  
 7   ticket    1309 non-null   object 
 8   fare      1308 non-null   float64
 9   embarked  1307 non-null   object 
dtypes: float64(2), int64(4), object(4)
memory usage: 102.4+ KB


In [6]:
# next... let's drop the columns that will probably not help us  ticket, fare, embarked, name
df = df.drop('ticket', axis=1)  
df = df.drop('fare', axis=1)  
df = df.drop('embarked', axis=1)  
df = df.drop('name', axis=1)  
df.info()  # re-look at the data ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   object 
 3   age       1046 non-null   float64
 4   sibsp     1309 non-null   int64  
 5   parch     1309 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 61.5+ KB


In [7]:
#
# drop all of the missing data (the "na" elements)
#
df = df.dropna()
df.info()   # re-look at the data...   (all are now 1043)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046 entries, 0 to 1308
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1046 non-null   int64  
 1   survived  1046 non-null   int64  
 2   sex       1046 non-null   object 
 3   age       1046 non-null   float64
 4   sibsp     1046 non-null   int64  
 5   parch     1046 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 57.2+ KB


In [8]:
#
# everything needs to be numeric, but the male/female ("sex") is a string... need to fix this!
#

def translate(s):
    """ from string to numeric value
    """
    d = { 'male':0, 'female':1 }
    return d[s]

df['sex'] = df['sex'].map(translate)     # this line converts from string to numeric value

df.info()   # re-look at the data...   (make sure all are numeric)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046 entries, 0 to 1308
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1046 non-null   int64  
 1   survived  1046 non-null   int64  
 2   sex       1046 non-null   int64  
 3   age       1046 non-null   float64
 4   sibsp     1046 non-null   int64  
 5   parch     1046 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 57.2 KB


In [9]:
#
# let's AUTOMATICALLY get the columns names in a list!
#
column_name_list = df.columns.values.tolist()
print(column_name_list)

['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch']


In [10]:
#
# let's convert our data to a numpy array, named A
#
A = df.values
print(A)

[[ 3.  -1.   0.  24.   0.   0. ]
 [ 3.  -1.   0.  25.   0.   0. ]
 [ 3.  -1.   0.  35.   0.   0. ]
 ...
 [ 3.   0.   0.  26.5  0.   0. ]
 [ 3.   0.   0.  27.   0.   0. ]
 [ 3.   0.   0.  29.   0.   0. ]]


In [11]:
print("+++ Start of data definitions +++\n")

# if we need to drop a column in the middle, we have to "restack" the arrays together:
X_all = np.hstack( (A[:,0:1], A[:,2:]) )  # X (features) ... is all rows, columns 0, not_1, 2, 3, 4, 5
y_all = A[:,1]                            # y (labels) ... is all rows, column 1 only

print("X_all is", X_all, X_all.shape)
print("y_all is", y_all)

+++ Start of data definitions +++

X_all is [[ 3.   0.  24.   0.   0. ]
 [ 3.   0.  25.   0.   0. ]
 [ 3.   0.  35.   0.   0. ]
 ...
 [ 3.   0.  26.5  0.   0. ]
 [ 3.   0.  27.   0.   0. ]
 [ 3.   0.  29.   0.   0. ]] (1046, 5)
y_all is [-1. -1. -1. ...  0.  0.  0.]


In [12]:
# column order is captured in these two "Python dictionaries":

NUM_TO_SPECIES = { 0:"perished", 1:"survived" }

SPECIES_TO_NUM = { "perished":0, "survived":1 }

COLS = {               # also: FEATURES
    "pclass":0,
    "sex":1,           # 'survived' has been removed: it's the y_all (labels)
    "age":2,
    "sibsp":3,         # should use this horizontal space more effectively!
    "parch":4,
}
# test this
print("Test COLS:  COLS['age'] is", COLS['age'])

# reverse the key/value ordering
COL_NAMES = {}
for key in COLS:
    value = COLS[key]
    COL_NAMES[value] = key # reversed!

print()   # let's test the reversed dictionary...
print("Test COL_NAMES:   COL_NAMES[0] is", COL_NAMES[0])

# let's use the dictionary!
the_data_row = A[55]  # digit at index 901
print("the_data_row is", the_data_row)
print()
print("Their age is ", the_data_row[COLS['age']])
print()
print("Simpler to type: ", the_data_row[3])

Test COLS:  COLS['age'] is 2

Test COL_NAMES:   COL_NAMES[0] is pclass
the_data_row is [ 1.  1.  1. 26.  0.  0.]

Their age is  1.0

Simpler to type:  26.0


In [ ]:
#
# the remainder of the workflow is for you to build!
#
#   [1] copy-and-paste is your friend!   HOWEVER,
#
#   [2] here, our goal is to understand what each cell is doing (by trying it out!!)
#

In [13]:
print(y_all[:45])

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1.  0.]


In [14]:
#
# break our dataset into distinct pieces:   unknown vs labeled
#

NUM_UNKNOWN = 42  # 42 unknown

X_unknown = X_all[:NUM_UNKNOWN,:]  # unknown up to here...
y_unknown = y_all[:NUM_UNKNOWN]    # unknown up to here...

X_labeled_orig = X_all[NUM_UNKNOWN:,:]  # labeled data starts at this index
y_labeled_orig = y_all[NUM_UNKNOWN:]    # labeled data starts at this index

#
# we scramble the data - but _only_ the labeled data!
# 
indices = np.random.permutation(len(y_labeled_orig))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_labeled_orig[indices]              # we must apply the _same_ permutation to each!
y_labeled = y_labeled_orig[indices]              # again...
print(y_labeled)

[0. 1. 1. ... 0. 1. 0.]


In [30]:
#
# we can re-weight different features here...
#

COL_WEIGHT = {              # also: FEATURE_WEIGHT  compare:  COLS
    'age':1.0,
    'pclass':10.0,
    'sex':10.0,
    'sibsp':1.0,
    'parch':1.0,
}

for col_name in COL_WEIGHT:
    i = COLS[col_name]    # get the column index, i, of the col_name
    weight = COL_WEIGHT[col_name]
    print("Weighting", col_name, "by", weight)   # weighting == "multiplying"
    
    X_labeled[:,i] *= weight      # multiply by the weight to give this column ("feature")


Weighting age by 1.0
Weighting pclass by 10.0
Weighting sex by 10.0
Weighting sibsp by 1.0
Weighting parch by 1.0


In [31]:
#
# the rest of the cells are similar to all of the others...
#

# you'll want to grab those...

In [32]:
# +++ This is the "Model-building and -training Cell"
#       
# Create a kNN model and train it! 
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ train on the testing data.
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# most common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#
NUM_ROWS = X_labeled.shape[0]     # the number of labeled rows
TEST_PERCENT = 0.20
TEST_SIZE = int(TEST_PERCENT*NUM_ROWS)   # no harm in rounding down

X_test = X_labeled[:TEST_SIZE]    # first section are for testing
y_test = y_labeled[:TEST_SIZE]

X_train = X_labeled[TEST_SIZE:]   # all the rest are for training
y_train = y_labeled[TEST_SIZE:]

print("testing with", TEST_SIZE, "rows; training with", (NUM_ROWS-TEST_SIZE), "rows")

testing with 200 rows; training with 804 rows


In [33]:
y_test # parished or survived for each passenger

array([0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.])

In [34]:
# +++ This is the "Model-building and -training Cell"
#       
# Create a kNN model and train it! 

from sklearn.neighbors import KNeighborsClassifier

k = 71   # we don't know what k to use, so we guess!  (71 will _not_ be a good value)
knn_model = KNeighborsClassifier(n_neighbors=k)   # here, k is the "k" in kNN

# we train the model (one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", k)  


Created and trained a knn classifier with k = 71


In [35]:
# +++ This is the "Model-testing Cell"
#
# Now, let's see how well we did on our "held-out data" (the testing data)
predicted_labels = knn_model.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

Predicted labels: [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 0. 0. 0. 1.]
Actual labels: [0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0

In [36]:
#
# Let's print these more helpfully, in a vertical table
#
#
# Let's print these more helpfully, in a vertical table
#

def compare_labels(predicted_labels, actual_labels):
    """ a nicer comparison, including counting how many are correct """
    NUM_LABELS = predicted_labels.shape[0]
    num_correct = 0
    
    for i in range(NUM_LABELS):
        p = predicted_labels[i]
        a = actual_labels[i]
        p_label = NUM_TO_SPECIES[p]
        a_label = NUM_TO_SPECIES[a]
        if p == a:
            toshow = ""
            num_correct += 1
        else:
            toshow = "incorrect"
            
        print(f"row {i}    |  {p}  {a}  |   {p_label} {a_label}   {toshow}")   # cool! "f-string" printing
        
        # even spiffier "formatted printing"  (cool stuff!)  
        #print(f"row {i:2d}    |  {p:1.0f}  {a:1.0f}  |   {p_label:>15s} {a_label:<15s}   {toshow:<15s}")
        
    # the loop is now complete...
    print()
    print("Correct:", num_correct, "out of", NUM_LABELS)
        
        
compare_labels(predicted_labels,actual_labels)


row 0    |  0.0  0.0  |   perished perished   
row 1    |  1.0  1.0  |   survived survived   
row 2    |  0.0  1.0  |   perished survived   incorrect
row 3    |  1.0  1.0  |   survived survived   
row 4    |  0.0  0.0  |   perished perished   
row 5    |  1.0  1.0  |   survived survived   
row 6    |  0.0  0.0  |   perished perished   
row 7    |  0.0  1.0  |   perished survived   incorrect
row 8    |  0.0  0.0  |   perished perished   
row 9    |  0.0  0.0  |   perished perished   
row 10    |  0.0  0.0  |   perished perished   
row 11    |  1.0  1.0  |   survived survived   
row 12    |  0.0  1.0  |   perished survived   incorrect
row 13    |  0.0  0.0  |   perished perished   
row 14    |  0.0  1.0  |   perished survived   incorrect
row 15    |  1.0  1.0  |   survived survived   
row 16    |  0.0  0.0  |   perished perished   
row 17    |  1.0  0.0  |   survived perished   incorrect
row 18    |  1.0  1.0  |   survived survived   
row 19    |  0.0  1.0  |   perished survived   incorr

In [41]:
#
# ok!  We _could_ be finished -- except that we _guessed_ at the keighborhood size (we guessed a k of 71)
#      Perhaps we should _explore_ whether other neighborhood sizes might better reflect our data... (?)

#
# to do this, we use "cross validation":  we split the _TRAINING_ data up and try different values of k
#
#
# ok!  We _could_ be finished -- except that we _guessed_ at the keighborhood size (we guessed a k of 71)
#      Perhaps we should _explore_ whether other neighborhood sizes might better reflect our data... (?)

#
# to do this, we use "cross validation":  we split the _TRAINING_ data up and try different values of k
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
# Lab challenge:  [1] loop over all values of k (or all _odd_ values of k...)  
#                 [2] assign best_k to the best one (highest average CV accuracy)

for k in range(1,72):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build knn_model with current value of k

    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # 5 is the "# of splits" (5 -> 80/20)
    #print('The 5 cross-validation accuracies: ', cv_scores,"\n")
    av = cv_scores.mean()
    print(f'k: {k:2d} produces an average CV accuracy of ', av)

# assign best value of k to best_k
best_k = 4      # at the moment, by hand - should be programmatic

k:  1 produces an average CV accuracy of  0.7487344720496895
k:  2 produces an average CV accuracy of  0.7898214285714287
k:  3 produces an average CV accuracy of  0.7947748447204969
k:  4 produces an average CV accuracy of  0.8121894409937888
k:  5 produces an average CV accuracy of  0.8084782608695653
k:  6 produces an average CV accuracy of  0.8010093167701863
k:  7 produces an average CV accuracy of  0.7960403726708074
k:  8 produces an average CV accuracy of  0.7935403726708075
k:  9 produces an average CV accuracy of  0.7985248447204969
k: 10 produces an average CV accuracy of  0.8022670807453416
k: 11 produces an average CV accuracy of  0.7960403726708074
k: 12 produces an average CV accuracy of  0.8034782608695652
k: 13 produces an average CV accuracy of  0.8022515527950311
k: 14 produces an average CV accuracy of  0.8035093167701863
k: 15 produces an average CV accuracy of  0.8084860248447205
k: 16 produces an average CV accuracy of  0.8022515527950311
k: 17 produces an averag

In [42]:
# print(f"k = {best_k}   yields the highest average CV accuracy.")  # print the best one
print(f"k = {best_k}   yields the highest average CV accuracy.")  # print the best one

k = 4   yields the highest average CV accuracy.


In [43]:
# Now, we re-create and re-run the  "Model-building and -training Cell"
#
# Now, using best_k instead of the original, randomly-guessed value    How does it do?!
# Now, we re-create and re-run the  "Model-building and -training Cell"
#
# Now, using best_k instead of the original, randomly-guessed value    How does it do?!

from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k

# we train the model (one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", best_k) 


Created and trained a knn classifier with k = 4


In [44]:
# And, we re-create and re-run the  "Model-testing Cell"
#
# Now, using best_k instead of the original, randomly-guessed value    How does it do?!

# And, we re-create and re-run the  "Model-testing Cell"
#
# Now, using best_k instead of the original, randomly-guessed value    How does it do?!

# Now, run our test set!
predicted_labels = knn_model.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# and, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 0. 0. 1. 1.]
Actual labels: [0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0

In [ ]:
#
# Plus, if new data arrives, you have a model with which to make predictions!
#
# here is a function for running new data (new flowers):
#


In [45]:
#
# test it out - with our own, typed data...
#

def predict_new_data(flower_data, knn_model):
    """ allows the user to enter values, from file or by hand, and then predict 
        the label using the knn_model passed in as input
    """
    # convert flower_data to a numpy array of float64 type (in case it's not already)
    flower_data = np.asarray(flower_data)
    flower_data.astype('float64')
    # how many dimensions?
    shape = flower_data.shape
    if len(shape) == 1:  # only one flower in one dimension ([a,b,c,d]) We need [[a,b,c,d]]
        flower_data = np.asarray( [flower_data] )  # need to wrap in extra square brackets!
        
    # print(flower_data.shape)
    NUM_ROWS, NUM_COLS = flower_data.shape   # get number of rows and number of cols

    # make the predictions
    predictions = knn_model.predict(flower_data)
    
    print()
    
    # print results
    for i in range(NUM_ROWS):
        flower = flower_data[i]
        predicted_label_as_number = predictions[i]
        predicted_label = NUM_TO_SPECIES[predicted_label_as_number]  # convert (back) to string!
        print(f"For {flower} the model predicts...   {predicted_label}")
        
    return predictions  # in case they're needed

print("predict_new_data defined.")

predict_new_data defined.


In [46]:
predictions = predict_new_data ( X_unknown, knn_model )


For [ 3.  0. 24.  0.  0.] the model predicts...   perished
For [ 3.  0. 25.  0.  0.] the model predicts...   survived
For [ 3.  0. 35.  0.  0.] the model predicts...   survived
For [ 3.  0. 18.  0.  0.] the model predicts...   perished
For [ 3.  1. 19.  1.  0.] the model predicts...   perished
For [ 3.  0. 32.  0.  0.] the model predicts...   perished
For [ 1.  0. 24.  0.  1.] the model predicts...   perished
For [ 1.  1. 50.  0.  1.] the model predicts...   perished
For [2. 0. 1. 2. 1.] the model predicts...   survived
For [2. 1. 4. 2. 1.] the model predicts...   survived
For [ 2.  1. 12.  2.  1.] the model predicts...   survived
For [ 2.  1. 36.  0.  3.] the model predicts...   perished
For [ 2.  0. 34.  0.  0.] the model predicts...   perished
For [ 2.  1. 19.  0.  0.] the model predicts...   perished
For [ 2.  0. 23.  0.  0.] the model predicts...   perished
For [ 2.  0. 26.  0.  0.] the model predicts...   survived
For [ 2.  0. 42.  0.  0.] the model predicts...   perished
For [ 